## Readme
1. 建立文件夹
2. 复制模型以及 Geometry 文件
   1. 先判断文件是否存在再复制

3. 定义传感器编号和环节的对应关系
4. 将 LMPS 坐标转换为 OpenSense 坐标 --》查找先前代码
5. 将 LMPS 文件转换为 .sto 文件

6. 运行 OpenSense 代码 --》查找先前代码
7. 在 OpenSim 中打开逆运动学结果

In [1]:
# 读取相对应的包
import os
import shutil
import pandas as pd

In [2]:
dir_path = "/Users/wangshuaibo/Library/CloudStorage/OneDrive-bsu.edu.cn/Archive/代码_OpenSense/OpenSense_Python/OpenSenseExample"
opensense_example_dir = "OpenSenseExample_Original"

In [4]:
#newfile_name = input("请输入新建文件夹名称：")
newfile_name = "20230615_LMPS_NoTorsoRightFeet"
newfile_path = os.path.join(dir_path, newfile_name)

In [5]:
# 新建文件夹
os.makedirs(newfile_path,exist_ok=True)

In [6]:
# 复制模型
src_Geometry = os.path.join(dir_path,opensense_example_dir,"Geometry")
dst_Geometry = os.path.join(dir_path,newfile_name,"Geometry")

src_model = os.path.join(dir_path,opensense_example_dir,"Rajagopal_2015.osim")
dst_model = os.path.join(dir_path,newfile_name,"Rajagopal_2015.osim")

shutil.copytree(src_Geometry, dst_Geometry,dirs_exist_ok=True)
shutil.copy2(src_model, dst_model)

'/Users/wangshuaibo/Library/CloudStorage/OneDrive-bsu.edu.cn/Archive/代码_OpenSense/OpenSense_Python/OpenSenseExample/20230615_LMPS_NoTorsoRightFeet/Rajagopal_2015.osim'

In [7]:
IMUData_dir = "IMUData"
file_name = "squat_100hz.csv"
file_path = os.path.join(dir_path,newfile_name,IMUData_dir,file_name)

In [8]:
df = pd.read_csv(file_path)
df.columns

Index(['SensorId', ' TimeStamp (s)', ' FrameNumber', ' AccX (g)', ' AccY (g)',
       ' AccZ (g)', ' GyroX (deg/s)', ' GyroY (deg/s)', ' GyroZ (deg/s)',
       ' MagX (uT)', ' MagY (uT)', ' MagZ (uT)', ' EulerX (deg)',
       ' EulerY (deg)', ' EulerZ (deg)', ' QuatW', ' QuatX', ' QuatY',
       ' QuatZ', ' LinAccX (g)', ' LinAccY (g)', ' LinAccZ (g)',
       ' Pressure (kPa)', ' Altitude (m)', ' Temperature (degC)',
       ' HeaveMotion (m)'],
      dtype='object')

In [13]:
# 定义四元数相乘函数
def quaternion_multiply(q1, q2):
    w1, x1, y1, z1 = q1
    w2, x2, y2, z2 = q2

    w = w1 * w2 - x1 * x2 - y1 * y2 - z1 * z2
    x = w1 * x2 + x1 * w2 + y1 * z2 - z1 * y2
    y = w1 * y2 - x1 * z2 + y1 * w2 + z1 * x2
    z = w1 * z2 + x1 * y2 - y1 * x2 + z1 * w2

    return (w, x, y, z)

In [41]:
# 转换小蓝块坐标为opensense 坐标系格式
import numpy as np 

# 定义数值
quats = [' QuatW', ' QuatX', ' QuatY', ' QuatZ']
quats_value = df[quats].values

# 将小蓝块东北天(ENU)坐标系转换到北西天(NWU)坐标系
# 全局坐标系即沿着 Z 轴旋转90度，数值为正
# 此处应该为静态旋转即外旋？因而应该是右乘？
angle_NWU2ENU = np.pi/2
q_NWU2ENU = (np.cos(angle_NWU2ENU),0,0,np.sin(angle_NWU2ENU))
result_NWU2ENU = quaternion_multiply(q_NWU2ENU,quats_value.T)

# 将传感器方向转换为 Xsens 规定方向
# 传感器绕着 y 轴旋转 180度
angle_opensense = np.pi  # 180度等于π弧度
q_y180 = (np.cos(angle_opensense / 2), 0, np.sin(angle_opensense / 2), 0)
result_opensense = quaternion_multiply(result_NWU2ENU, q_y180)

# 将结果转换为DataFrame
tran_quats = ["trans q0","trans q1","trans q2","trans q3"]
df[tran_quats] = pd.DataFrame(list(result_opensense)).transpose()

In [43]:
bodies = ["pelvis","tibia_r","femur_r","tibia_l",'femur_l','calcn_l']
body_data = pd.DataFrame()

body = "pelvis"
body_quars = [body + " " + quat for quat in tran_quats]
body_quars

['pelvis trans q0', 'pelvis trans q1', 'pelvis trans q2', 'pelvis trans q3']

In [50]:
df[df['SensorId'] == 5]

,SensorId,TimeStamp (s),FrameNumber,AccX (g),AccY (g),AccZ (g),GyroX (deg/s),GyroY (deg/s),GyroZ (deg/s),MagX (uT),...,Temperature (degC),HeaveMotion (m),tran_qw,tran_qx,tran_qy,tran_qz,trans q0,trans q1,trans q2,trans q3
0,5,0.00,0,-0.985503,0.220812,-0.134935,0.288531,-0.166540,-0.057938,-14.820000,...,0.0,0.0,0.2773,-0.4785,0.5584,-0.6182,0.2773,-0.4785,0.5584,-0.6182
18,5,0.01,1,-0.986513,0.217862,-0.137964,0.284083,0.115996,-0.112218,-15.309999,...,0.0,0.0,0.2772,-0.4793,0.5577,-0.6182,0.2772,-0.4793,0.5577,-0.6182
19,5,0.02,2,-0.984494,0.219840,-0.136951,0.513701,0.061821,-0.169192,-15.309999,...,0.0,0.0,0.2770,-0.4801,0.5571,-0.6182,0.2770,-0.4801,0.5571,-0.6182
20,5,0.03,3,-0.984494,0.219840,-0.136951,0.685378,0.118696,-0.110827,-14.790000,...,0.0,0.0,0.2769,-0.4809,0.5565,-0.6182,0.2769,-0.4809,0.5565,-0.6182
36,5,0.04,4,-0.984479,0.218825,-0.133952,0.398559,0.061708,-0.226777,-14.820000,...,0.0,0.0,0.2768,-0.4817,0.5559,-0.6182,0.2768,-0.4817,0.5559,-0.6182
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5244,5,10.44,1044,-0.981521,0.222931,-0.150967,-0.241506,0.899810,0.065698,-15.780000,...,0.0,0.0,0.2322,-0.6298,0.4201,-0.6105,0.2322,-0.6298,0.4201,-0.6105
5245,5,10.45,1045,-0.983552,0.222942,-0.152966,-0.068244,0.843934,0.122900,-15.750000,...,0.0,0.0,0.2322,-0.6298,0.4202,-0.6104,0.2322,-0.6298,0.4202,-0.6104
5246,5,10.46,1046,-0.986574,0.217998,-0.155004,-0.582303,0.671993,0.062179,-15.200000,...,0.0,0.0,0.2321,-0.6299,0.4202,-0.6104,0.2321,-0.6299,0.4202,-0.6104
5247,5,10.47,1047,-0.987587,0.218978,-0.153991,-0.237850,0.673650,0.120559,-15.719999,...,0.0,0.0,0.2321,-0.6300,0.4202,-0.6103,0.2321,-0.6300,0.4202,-0.6103


In [53]:
freq = df["SensorId"].value_counts

In [54]:
freq

<bound method IndexOpsMixin.value_counts of 0       5
1       2
2       2
3       2
4       2
       ..
5244    5
5245    5
5246    5
5247    5
5248    5
Name: SensorId, Length: 5249, dtype: int64>